<a href="https://colab.research.google.com/github/RAFS20/Ricardo/blob/master/Modelo_de_ciudades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de modelo de ciudades


In [1]:
import random

# Constantes
MAX_PXCOR = 50
MAX_PYCOR = 50
DENSIDAD_INICIAL = 0.05
CAPITAL_REQUERIDO = 10
TIEMPO_MIGRACION = 10
TIEMPO_CONSTRUCCION = 5
TIEMPO_CRUCE = 15


class Parcela:
    def __init__(self):
        self.valor_terreno = 0.0
        self.agua = 0.0
        self.electricidad = 0.0
        self.ocupado = "no"
        self.transport = 0.0
        self.ctr = ""
        self.elevacion = 9.9
        self.colonia = 0


class Tortuga:
    def __init__(self):
        self.tamaño = 0.0
        self.forma = ""
        self.color = ""
        self.rumbo = 0.0
        self.energia = 0.0
        self.tamaño_bloque = 0.0
        self.peso = 0.0
        self.tamaño_pluma = 0.0
        self.pluma_abajo = False
        self.demasiado_cerca = []
        self.origen = ""
        self.ingreso = 0.0
        self.ahorros = 0.0
        self.trabajos = []
        self.residente_desde = 0
        self.gastos_vivienda = 0.0
        self.etiqueta = ""
        self.tamaño_colonia = 0.0
        self.oculto = False
        self.crecimiento_ciudad = False
        self.cruzado = 0


class Constructor(Tortuga):
    def __init__(self):
        super().__init__()


class ConstructorSegundoOrden(Tortuga):
    def __init__(self):
        super().__init__()


class ConstructorTercerOrden(Tortuga):
    def __init__(self):
        super().__init__()


class Migrante(Tortuga):
    def __init__(self):
        super().__init__()


class Anillo(Tortuga):
    def __init__(self):
        super().__init__()


class Nodo(Tortuga):
    def __init__(self):
        super().__init__()


class CentroEmpleo(Tortuga):
    def __init__(self):
        super().__init__()
        self.tamaño = 0.0
        self.agua = 0.0
        self.electricidad = 0.0
        self.transport = 0.0


class CentroServicio(Tortuga):
    def __init__(self):
        super().__init__()
        self.valor_terreno = 0.0
        self.agua = 0.0
        self.electricidad = 0.0
        self.transport = 0.0


# Variables globales
valores = 0.0
pesos = 0.0
valor_marginal = 0.0
peso_marginal = 0.0
cruzado = 0
tercer_fase = ""
costo_max = 0.0
configuracion_hecha = False


def configuracion():
    global configuracion_hecha

    if configuracion_hecha:
        print("La configuración del paisaje urbano ya se ha completado.")
        return

    paso()
    generar_paisaje_urbano()

    if existen_centros_empleo() and not existen_migrantes():
        condicion_inicial()

    if existen_migrantes_con_tamaño(0.65):
        actualizar_valor_terrenos()
        condicion_inicial_2()

    if tercer_fase == "ir":
        economia()

    if tercer_fase == "detener":
        actualizar_valor_terrenos()
        reiniciar_pasos()
        configuracion_hecha = True


def generar_paisaje_urbano():
    if pasos == 1:
        generar_topografia()
        region_centro = obtener_parcelas_condicion(lambda pxcor, pycor: abs(pxcor) < (MAX_PXCOR - 20)
                                                                               and abs(pycor) < (MAX_PYCOR - 20))

        parcelas = []
        for pxcor in range(MAX_PXCOR):
            for pycor in range(MAX_PYCOR):
                parcela = Parcela()
                parcela.agua = 10.0
                parcela.electricidad = 1.0
                parcela.ocupado = "no"
                parcela.transport = 1.5
                parcelas.append(parcela)

        n_centros_servicio = len(region_centro)
        centros_servicio = []
        constructores = []
        for _ in range(n_centros_servicio):
            centro_servicio = CentroServicio()
            centro_servicio.valor_terreno = 35
            centros_servicio.append(centro_servicio)

            vecinos = obtener_vecinos(centro_servicio)
            for vecino in vecinos:
                vecino.valor_terreno = 35

            for parcela in obtener_parcelas_en_radio(centro_servicio, 20):
                parcela.elevacion = 9.9

            centro_servicio.agua = 2.0
            centro_servicio.electricidad = 3.0
            centro_servicio.transport = 1.0

            for vecino in vecinos:
                vecino.agua = 2.0
                vecino.electricidad = 3.0
                vecino.transport = 1.0

            constructor = Constructor()
            constructor.forma = "ojo"
            constructor.tamaño = 5
            constructor.color = 4
            constructores.append(constructor)

            constructor.origen = [constructor]
            centro_servicio.constructores = [constructor]

        hacia_otros_centros(constructores)

    for constructor in constructores:
        constructor.demasiado_cerca = []
        constructor.adelante(1)
        constructor.color = 6
        if pasos % 20 == 0:
            constructor.rumbo = normal_aleatorio(constructor.rumbo, 3)
        constructor.peso -= 0.02
        constructor.peso = round(constructor.peso, 3)
        constructor.valor_terreno = constructor.peso * 12
        constructor.transport = 0.75

        for parcela in obtener_parcelas_en_radio(constructor, 8):
            parcela.ctr = "+"

    for constructor in constructores:
        lista_pasos = [n * constructor.tamaño_bloque for n in [1, 3, 7, 10]]
        if pasos in lista_pasos:
            generar_constructores_segundo_orden()

        if constructor.parcela_adelante(2) is None:
            constructor.morir()

        if constructor.parcela_adelante(2).transporte == 0.75:
            constructor.adelante(2)
            constructor.morir()

    limite_fase_1 = round(normal_aleatorio(100, 10))

    for constructor_segundo_orden in constructores_segundo_orden:
        constructor_segundo_orden.etiqueta = ""
        if constructor_segundo_orden.energia != 0:
            constructor_segundo_orden.adelante(0.5)
        constructor_segundo_orden.pluma_abajo = True
        constructor_segundo_orden.color = 7
        constructor_segundo_orden.tamaño_pluma = 1
        constructor_segundo_orden.peso -= 0.001
        constructor_segundo_orden.peso = round(constructor_segundo_orden.peso, 3)
        constructor_segundo_orden.valor_terreno = constructor_segundo_orden.peso * 7
        constructor_segundo_orden.agua = 2.0
        constructor_segundo_orden.electricidad = 3.0
        constructor_segundo_orden.transport = 1.0
        if constructor_segundo_orden.peso < 0:
            constructor_segundo_orden.peso = 0

        if round(constructor_segundo_orden.rumbo) % 15 < 5 or round(constructor_segundo_orden.rumbo) % 15 > 10:
            constructor_segundo_orden.rumbo += round(normal_aleatorio(5, 10))

        if constructor_segundo_orden.ctr == "+" and distancia(uno_de(centros_servicio), constructor_segundo_orden) < 60:
            constructor_segundo_orden.energia -= 0.5

        if constructor_segundo_orden.ctr != "+":
            constructor_segundo_orden.energia -= 1
            if distancia(uno_de(centros_servicio), constructor_segundo_orden) > 60:
                constructor_segundo_orden.energia -= 2

        if constructor_segundo_orden.energia < 0.5:
            constructor_segundo_orden.energia = 0

        if constructor_segundo_orden.parcela_adelante(1) is None:
            constructor_segundo_orden.morir()

        if constructor_segundo_orden.parcela_adelante(1).elevacion < 9.9:
            constructor_segundo_orden.adelante(0.5)
            constructor_segundo_orden.morir()

        if constructor_segundo_orden.parcela_adelante(1).valor_terreno != 0:
            constructor_segundo_orden.adelante(0.5)
            constructor_segundo_orden.morir()

        if constructor_segundo_orden.parcela_adelante(1).transporte == 0.75:
            constructor_segundo_orden.adelante(1)
            constructor_segundo_orden.morir()

        if pasos < limite_fase_1 and constructor_segundo_orden.energia == 0:
            constructor_segundo_orden.morir()

        if pasos % round(normal_aleatorio(5, 1)) == 0 and constructor_segundo_orden.energia != 0:
            eclosionar_constructores_tercer_orden(1, lambda constructor: constructor.derecha(90), lambda constructor: constructor.energia = normal_aleatorio(constructor.energia, 2) + 2)
            eclosionar_constructores_tercer_orden(1, lambda constructor: constructor.izquierda(90), lambda constructor: constructor.energia = normal_aleatorio(constructor.energia, 2) + 2)

    for constructor_tercer_orden in constructores_tercer_orden:
        constructor_tercer_orden.etiqueta = ""
        if constructor_tercer_orden.energia != 0:
            constructor_tercer_orden.adelante(0.5)
        constructor_tercer_orden.pluma_abajo = True
        constructor_tercer_orden.color = 7
        constructor_tercer_orden.tamaño_pluma = 1
        constructor_tercer_orden.peso -= 0.0007
        constructor_tercer_orden.peso = round(constructor_tercer_orden.peso, 3)
        constructor_tercer_orden.valor_terreno = constructor_tercer_orden.peso * 7
        constructor_tercer_orden.agua = 2.0
        constructor_tercer_orden.electricidad = 3.0
        constructor_tercer_orden.transport = 1.0
        if constructor_tercer_orden.peso > 0.3:
            for parcela in obtener_parcelas_en_radio(constructor_tercer_orden, 2):
                parcela.electricidad = 3.0

        if constructor_tercer_orden.peso < 0:
            constructor_tercer_orden.peso = 0

        if round(constructor_tercer_orden.rumbo) % 15 < 5 or round(constructor_tercer_orden.rumbo) % 15 > 10:
            constructor_tercer_orden.rumbo += round(normal_aleatorio(5, 10))

        if constructor_tercer_orden.ctr == "+" and distancia(uno_de(centros_servicio), constructor_tercer_orden) < 60:
            constructor_tercer_orden.energia -= 0.5

        if constructor_tercer_orden.ctr != "+":
            constructor_tercer_orden.energia -= 1

        if distancia(uno_de(centros_servicio), constructor_tercer_orden) > 60:
            constructor_tercer_orden.energia -= 2

        if constructor_tercer_orden.energia < 0.5:
            constructor_tercer_orden.energia = 0

        if pasos % round(normal_aleatorio(3.75, 0.25)) == 0 and constructor_tercer_orden.energia != 0:
            eclosionar(1, lambda constructor: constructor.derecha(90), lambda constructor: constructor.energia = normal_aleatorio(constructor.energia, 2) + 2)
            eclosionar(1, lambda constructor: constructor.izquierda(90), lambda constructor: constructor.energia = normal_aleatorio(constructor.energia, 2) + 2)

        if constructor_tercer_orden.parcela_adelante(1) is None:
            constructor_tercer_orden.morir()

        if constructor_tercer_orden.parcela_adelante(1).elevacion < 9.9:
            constructor_tercer_orden.adelante(0.5)
            constructor_tercer_orden.morir()

        if constructor_tercer_orden.parcela_adelante(1).valor_terreno != 0:
            constructor_tercer_orden.adelante(0.5)
            constructor_tercer_orden.morir()

        if constructor_tercer_orden.parcela_adelante(1).transporte == 0.75:
            constructor_tercer_orden.adelante(1)
            constructor_tercer_orden.morir()

        if constructor_tercer_orden.parcela_adelante(1).transporte != 0.75:
            constructor_tercer_orden.agua = 2.0

        if constructor_tercer_orden.parcela_adelante(1).transporte != 0.75:
            constructor_tercer_orden.electricidad = 3.0

        if constructor_tercer_orden.parcela_adelante(1).transporte != 0.75:
            constructor_tercer_orden.transport = 1.0

        if constructor_tercer_orden.parcela_adelante(1).transporte != 0.75:
            constructor_tercer_orden.adelante(0.5)

    for parcela in parcelas:
        if parcela.valor_terreno > 0:
            parcela.valor_terreno = parcela.valor_terreno

    actualizar_graficos()


def actualizar_graficos():
    graficar("Valor del Terreno", media([parcela.valor_terreno for parcela in parcelas]))
    graficar("Agua", media([parcela.agua for parcela in parcelas]))
    graficar("Electricidad", media([parcela.electricidad for parcela in parcelas]))
    graficar("Transporte", media([parcela.transport for parcela in parcelas]))
    graficar("Ingreso", media([tortuga.ingreso for tortuga in tortugas if isinstance(tortuga, (Constructor, ConstructorTercerOrden))]))


def hacia_otros_centros(constructores):
    for constructor in constructores:
        if constructor.peso > 0:
            for centro in uno_de(centros_servicio):
                constructor.hacia(centro)
                constructor.adelante(1)
                constructor.derecha(90)


def generar_constructores_segundo_orden():
    region_centro = obtener_parcelas_condicion(lambda pxcor, pycor: abs(pxcor) < (MAX_PXCOR - 20)
                                                                                   and abs(pycor) < (MAX_PYCOR - 20))

    constructores_segundo_orden = []
    for _ in range(len(region_centro)):
        constructor_segundo_orden = ConstructorSegundoOrden()
        constructor_segundo_orden.color = 7
        constructor_segundo_orden.constructores = constructores
        constructor_segundo_orden.agua = 2.0
        constructor_segundo_orden.electricidad = 3.0
        constructor_segundo_orden.transport = 1.0
        constructor_segundo_orden.rumbo = 0.0
        constructor_segundo_orden.peso = 0.3
        constructor_segundo_orden.valor_terreno = 2.1
        constructor_segundo_orden.demasiado_cerca = []

        constructores.append(constructor_segundo_orden)
        constructores_segundo_orden.append(constructor_segundo_orden)


def eclosionar_constructores_tercer_orden(n, funcion_rumbo, funcion_energia):
    for _ in range(n):
        constructor_tercer_orden = ConstructorTercerOrden()
        constructor_tercer_orden.constructores = constructores_segundo_orden
        constructor_tercer_orden.agua = 2.0
        constructor_tercer_orden.electricidad = 3.0
        constructor_tercer_orden.transport = 1.0
        constructor_tercer_orden.rumbo = 0.0
        constructor_tercer_orden.peso = 0.3
        constructor_tercer_orden.valor_terreno = 2.1
        constructor_tercer_orden.demasiado_cerca = []

        constructor_segundo_orden = uno_de(constructores_segundo_orden)
        constructor_tercer_orden.origen = [constructor_segundo_orden]

        constructor_tercer_orden.rumbo = constructor_segundo_orden.rumbo
        constructor_tercer_orden.adelante(1)
        funcion_rumbo(constructor_tercer_orden)
        funcion_energia(constructor_tercer_orden)

        constructores_tercer_orden.append(constructor_tercer_orden)


def economia():
    global tercer_fase
    tercer_fase = "detener"


def actualizar_valor_terrenos():
    for parcela in parcelas:
        if parcela.valor_terreno == 0:
            parcela.valor_terreno = 9.9


def condicion_inicial():
    for constructor in constructores:
        constructor.color = 5
        constructor.peso = 0.5
        constructor.valor_terreno = constructor.peso * 12
        constructor.transport = 1.5
        constructor.rumbo = 180


def condicion_inicial_2():
    for constructor in constructores:
        constructor.tamaño = 3.0
        constructor.color = 4
        constructor.valor_terreno = 3.6
        constructor.peso = 0.3
        constructor.transport = 1.0


def existen_centros_empleo():
    return len([tortuga for tortuga in tortugas if isinstance(tortuga, CentroEmpleo)]) > 0


def existen_migrantes():
    return len([tortuga for tortuga in tortugas if isinstance(tortuga, Migrante)]) > 0


def existen_migrantes_con_tamaño(tamaño):
    return any([tortuga for tortuga in tortugas if isinstance(tortuga, Migrante) and tortuga.tamaño >= tamaño])


def normal_aleatorio(mu, sigma):
    return random.normalvariate(mu, sigma)


def obtener_parcelas_condicion(funcion_condicion):
    return [parcela for parcela in parcelas if funcion_condicion(parcela.pxcor, parcela.pycor)]


def obtener_vecinos(tortuga):
    vecinos = []
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx != 0 or dy != 0:
                vecino = parcela_en(tortuga.xcor + dx, tortuga.ycor + dy)
                if vecino is not None:
                    vecinos.append(vecino)
    return vecinos


def obtener_parcelas_en_radio(tortuga, radio):
    parcelas_en_radio = []
    for dx in range(-radio, radio + 1):
        for dy in range(-radio, radio + 1):
            parcela = parcela_en(tortuga.xcor + dx, tortuga.ycor + dy)
            if parcela is not None:
                distancia = abs(dx) + abs(dy)
                if distancia <= radio:
                    parcelas_en_radio.append(parcela)
    return parcelas_en_radio


def media(valores):
    if len(valores) > 0:
        return sum(valores) / len(valores)
    else:
        return 0.0


# Ejemplo de uso
configuracion_hecha()


SyntaxError: ignored